# Example Notebook for DPNeurifyFV

To make sure the notebook runs with the `DPNeurifyFV` environment, activate the directory with the corresponding `Project.toml` in Julia's package manager.

**General note**: Julia precompiles a function, when it is first used. This may lead to the first execution to be rather slow. It should be much faster, for the second execution.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.status()

  Activating environment at `~/VerifyNN/DPNeurifyFV/Project.toml`


     Project DPNeurifyFV v0.1.0
      Status `~/VerifyNN/DPNeurifyFV/Project.toml`
  [864edb3b] DataStructures v0.18.11
  [b4f0291d] LazySets v1.56.3
  [7e1232c4] NeuralPriorityOptimizer v0.1.0 `https://github.com/sisl/NeuralPriorityOptimizer.jl#main`
  [146f25fa] NeuralVerification v0.1.0 `https://github.com/phK3/NeuralVerification.jl#BuildingBranch`
  [d96e819e] Parameters v0.12.3
  [37e2e46d] LinearAlgebra


Import the necessary packages `DPNeurifyFV`, `NeuralVerification` and `LazySets`.

In [48]:
using NeuralVerification, DPNeurifyFV, LazySets

We are going to demonstrate the functionality on the example of the first ACAS-Xu network operating on the input defined in property $\phi_1$.

After loading the network, we convert it into a format that stores the negative and positive weights separately to make the symbolic forward pass with `DPNeurifyFV` more efficient.

In [5]:
input_set, output_set = DPNeurifyFV.get_acas_sets(1)

acas = read_nnet("./networks/ACASXU_experimental_v2a_1_1.nnet")
acas_npi = NetworkNegPosIdx(acas);

## Single Forward Pass

We demonstrate the effect of introducing the maximum number of fresh variables (the original idea is given in the [NeuroDiff paper](https://arxiv.org/abs/2009.09943)) at the first possible neurons and how it can be improved by 
- introducing at fresh variables for at most a certain fraction of the neurons in a layer and
- selecting the neurons for which to introduce fresh variables by the range between their concrete lower and upper bounds.

The effect of the techniques is visible as the computed lower and upper bounds of the $5$ output neurons of the network get progressively tighter.

Bounds computed by zonotope propagation are also shown for comparison.

In [25]:
max_vars = 0
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars);
ŝ = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s);
[ŝ.lbs[end] ŝ.ubs[end]]

5×2 Matrix{Float64}:
  -945.744  2627.28
 -1593.13   3431.0
 -1107.33   3485.83
 -2657.43   3830.79
 -1721.52   3815.88

In [45]:
max_vars = 20  # best possible value found with linear search
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars);
ŝ = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s);
[ŝ.lbs[end] ŝ.ubs[end]]

5×2 Matrix{Float64}:
  -713.548  1986.05
 -1206.81   2595.46
  -836.401  2635.62
 -2011.3    2896.48
 -1298.61   2885.86

In [23]:
max_vars = 43  # best possible value found with linear search
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars);
ŝ = forward_network(DPNFV(method=:DeepPolyRelax, get_fresh_var_idxs=DPNeurifyFV.fresh_var_range_non_zero, var_frac=0.2), acas_npi, s);
[ŝ.lbs[end] ŝ.ubs[end]]

5×2 Matrix{Float64}:
  -545.844  1327.61
  -849.924  1711.73
  -620.444  1682.68
 -1369.79   1985.86
  -968.068  1921.66

In [49]:
# Zonotope propagation
ẑ = forward_network(Ai2z(), acas, input_set)
[low(ẑ) high(ẑ)]

5×2 Matrix{Float64}:
 -3284.19  5023.29
 -3858.88  5701.12
 -3974.83  6211.21
 -4135.37  5365.68
 -3829.93  5514.25

# Branch and Bound with Input Splitting

We are now going to maximize the first output of the network over the input space given by property $\phi_1$.

In [51]:
params = DPNeurifyFV.PriorityOptimizerParameters(max_steps=5000, print_frequency=100, stop_frequency=1, verbosity=2)
optimize_linear_deep_poly(acas, input_set, [1.,0,0,0,0], params, solver=DPNFV(method=:DeepPolyRelax, max_vars=15), concrete_sample=:BoundsMaximizer, split=DPNeurifyFV.split_important_interval)

i: 100 - [-0.017807535560300215, 0.28346228356969677], 0.1857128143310547 sec
i: 200 - [-0.017807535560300215, -0.0005626577176235797], 0.3333280086517334 sec
i: 300 - [-0.017807535560300215, -0.014229881669890589], 0.47218894958496094 sec
i: 400 - [-0.017807535560300215, -0.01616575874515486], 0.6078557968139648 sec
i: 500 - [-0.017768583207969105, -0.01700112607245889], 0.7410039901733398 sec
i: 600 - [-0.01771721730558143, -0.017293899238105828], 0.8720948696136475 sec
i: 700 - [-0.01768762570130527, -0.017444203723618303], 0.998323917388916 sec
i: 800 - [-0.01768762570130527, -0.017532476976106312], 1.119107961654663 sec
i: 900 - [-0.01768762570130527, -0.01757186146113745], 1.237293004989624 sec
i: 1000 - [-0.01768762570130527, -0.017591160922579428], 1.3489699363708496 sec
i: 1059 - [-0.01768762570130527, -0.017602731762479756], 1.42289400100708 sec


([0.5999999999999999, 0.0009765625, 0.34375, 0.5, -0.45], -0.01768762570130527, -0.017602731762479756, 1059)